In [1]:
from pprint import pprint

In [2]:
from workflowsteps import *

## setup

In [3]:
idd_path = r"..\demo\Energy+V9_0_1.idd"
idf_path = r"..\demo\option1\ASHRAE901_SchoolPrimary_STD2004_ElPaso.idf"
idf_path_out = r"..\demo\option1\ASHRAE901_SchoolPrimary_STD2004_ElPaso_injected.idf"
weather_path = r"..\weather\USA_TX_El.Paso.Intl.AP.722700_TMY3.epw"
windows_demo_foler = r"..\demo\option1"

In [4]:
items = read_json_file("../schema/item2_poc.json")

**items[0] is:**

```json
{
    "no": 1,
    "verification_type": "compliance",
    "description_brief": "Economizer Integration: Non-Integrated Economizer Operation",
    "description_detail": "Economizer systems shall be integrated with the mechanical cooling system and be capable of providing partial cooling even when additional mechanical cooling is required to meet the remainder of the cooling load.",
    "description_index": "Section 6.5.1.3 in 90.-2004",
    "simulation_period": "annual",
    "datapoints_source": {
        "idf_objects": {
            "OA_min_sys": {
                "filters": {
                    "idf_object_type": "Controller:OutdoorAir",
                    "Name": "PSZ-AC_1:6_OA_Controller"
                },
                "exclusions": {"Minimum_Outdoor_Air_Flow_Rate": "Autosize"},
                "field": "Minimum_Outdoor_Air_Flow_Rate"
            }
        },
        "idf_output_variables": {
            "OA_timestep": {
                "subject": "PSZ-AC_1:6_OAInlet Node",
                "variable": "System Node Standard Density Volume Flow Rate",
                "frequency": "timestep"
            },
            "Cool_sys_out": {
                "subject": "PSZ-AC_1:6",
                "variable": "Air System Cooling Coil Total Cooling Energy",
                "frequency": "timestep"
            }
        },
        "parameters": {},
        "dev_settings": {}
    },
    "description_datapoints": {
        "OA_timestep": "System OA for a particular timestep, simulation output",
        "Cool_sys_out": "System/air-loop cooling coil output (could be + or -, check EnergyPlus I/O guide), simulation output",
        "OA_min_sys": "Minimum system OA, simulation output"
    },
    "assertions_type": "fail",
    "assertion_level": "sample",
    "assertions": ["$OA_timestep > $OA_min_sys and $Cool_sys_out > 0"]
}
```

## Workflow

### Read verification item

In [5]:
item = build_an_item(items[0])

idf_objects
{'OA_min_sys': {'filters': {'idf_object_type': 'Controller:OutdoorAir', 'Name': 'PSZ-AC_1:6_OA_Controller'}, 'exclusions': {'Minimum_Outdoor_Air_Flow_Rate': 'Autosize'}, 'field': 'Minimum_Outdoor_Air_Flow_Rate'}}
idf_output_variables
{'OA_timestep': {'subject': 'PSZ-AC_1:6_OAInlet Node', 'variable': 'System Node Standard Density Volume Flow Rate', 'frequency': 'timestep'}, 'Cool_sys_out': {'subject': 'PSZ-AC_1:6', 'variable': 'Air System Cooling Coil Total Cooling Energy', 'frequency': 'timestep'}}
parameters
{}
dev_settings
{}


In [6]:
idf_outputs = read_injection_points(item)
idf_outputs

In [7]:
print(
    f"A total of {len(idf_outputs)} EP Output variables objecs are extracted from {len(items)} item(s)"
)

A total of 2 EP Output variables objecs are extracted from 1 item(s)


In [8]:
unique_output = combine_injection_points(idf_outputs)
print(
    f"{len(unique_output)} unique output variables will be added to file"
)

2 unique output variables will be added to file


### Inject model

In [9]:
!dir $windows_demo_foler

 Volume in drive C is Windows
 Volume Serial Number is A84E-D40D

 Directory of C:\GitRepos\hvac-verify\demo\option1

09/15/2020  04:58 PM    <DIR>          .
09/15/2020  04:58 PM    <DIR>          ..
02/19/2020  07:28 PM           876,595 ASHRAE901_SchoolPrimary_STD2004_ElPaso.idf
               1 File(s)        876,595 bytes
               2 Dir(s)  724,336,541,696 bytes free


In [10]:
inject_idf(
    iddpath=idd_path,
    idfpath_in=idf_path,
    objstoinject=unique_output,
    idfpath_out=idf_path_out,
)

In [11]:
!dir $windows_demo_foler

 Volume in drive C is Windows
 Volume Serial Number is A84E-D40D

 Directory of C:\GitRepos\hvac-verify\demo\option1

09/15/2020  04:59 PM    <DIR>          .
09/15/2020  04:59 PM    <DIR>          ..
02/19/2020  07:28 PM           876,595 ASHRAE901_SchoolPrimary_STD2004_ElPaso.idf
09/15/2020  04:59 PM           877,048 ASHRAE901_SchoolPrimary_STD2004_ElPaso_injected.idf
               2 File(s)      1,753,643 bytes
               2 Dir(s)  724,335,558,656 bytes free


### run simulation

In [12]:
run_simulation(
    idfpath=idf_path_out,
    weatherpath=weather_path,
)
print("Workflow: simulation complete")

Running simulation: ..\demo\option1\ASHRAE901_SchoolPrimary_STD2004_ElPaso_injected.idf -- 2020-09-15 16:59:09.803575
Simulation complete -- 2020-09-15 17:03:32.640572
Simulation log saved to output folder -- 2020-09-15 17:03:32.649574
Workflow: simulation complete


In [13]:
sim_folder = idf_path_out[:-4]
!dir $sim_folder

 Volume in drive C is Windows
 Volume Serial Number is A84E-D40D

 Directory of C:\GitRepos\hvac-verify\demo\option1\ASHRAE901_SchoolPrimary_STD2004_ElPaso_injected

09/15/2020  05:03 PM    <DIR>          .
09/15/2020  05:03 PM    <DIR>          ..
09/15/2020  05:03 PM           472,313 eplusmtr.csv
09/15/2020  05:03 PM             1,721 eplusout.audit
09/15/2020  05:03 PM           250,617 eplusout.bnd
09/15/2020  05:03 PM         2,327,457 eplusout.csv
09/15/2020  04:59 PM           105,461 eplusout.dxf
09/15/2020  05:03 PM           305,556 eplusout.eio
09/15/2020  05:03 PM               102 eplusout.end
09/15/2020  05:03 PM            23,701 eplusout.err
09/15/2020  05:03 PM         4,002,668 eplusout.eso
09/15/2020  04:59 PM            37,296 eplusout.mdd
09/15/2020  05:03 PM           180,122 eplusout.mtd
09/15/2020  05:03 PM           725,841 eplusout.mtr
09/15/2020  04:59 PM            65,561 eplusout.rdd
09/15/2020  05:03 PM             1,034 eplusout.rvaudit
09/15/2020  04:59

### Read simulation i/o required by verification

In [14]:
df = read_points(
    runpath=sim_folder,
    idf_filename="in.idf",
    csv_filename="eplusout.csv",
    idd_path=idd_path,
    item=item,
)

2
2
['Date/Time', 'PSZ-AC_1:6_OAINLET NODE:System Node Standard Density Volume Flow Rate [m3/s](TimeStep)', 'PSZ-AC_1:6:Air System Cooling Coil Total Cooling Energy [J](TimeStep) ']


In [15]:
df.head()

,Date/Time,OA_timestep,Cool_sys_out,OA_min_sys
0,01/01 00:10:00,0.112816,0.0,0.0
1,01/01 00:20:00,0.106475,0.0,0.0
2,01/01 00:30:00,0.067460,0.0,0.0
3,01/01 00:40:00,0.000000,0.0,0.0
4,01/01 00:50:00,0.116668,0.0,0.0


### Run verification

In [16]:
results, testerdf = run_rules(item, df)

In [17]:
testerdf[testerdf["$OA_timestep > $OA_min_sys and $Cool_sys_out > 0"] == True][testerdf.columns[0:4]]

,Date/Time,OA_timestep,Cool_sys_out,OA_min_sys
180,01/02 06:10:00,0.334317,3.308728e+06,0.0
186,01/02 07:10:00,0.336712,8.109516e+02,0.0
187,01/02 07:20:00,0.342238,6.792602e+02,0.0
188,01/02 07:30:00,0.347249,3.332488e+02,0.0
189,01/02 07:40:00,0.357945,1.496041e+03,0.0
...,...,...,...,...
52171,12/29 07:20:00,0.239365,4.670565e+02,0.0
52172,12/29 07:30:00,0.238161,4.851751e+02,0.0
52173,12/29 07:40:00,0.241309,2.072151e+02,0.0
52174,12/29 07:50:00,0.244758,1.443468e+03,0.0


### Verification results

In [18]:
testerdf[testerdf["$OA_timestep > $OA_min_sys and $Cool_sys_out > 0"] == False][testerdf.columns[0:4]]

,Date/Time,OA_timestep,Cool_sys_out,OA_min_sys
0,01/01 00:10:00,0.112816,0.0,0.0
1,01/01 00:20:00,0.106475,0.0,0.0
2,01/01 00:30:00,0.067460,0.0,0.0
3,01/01 00:40:00,0.000000,0.0,0.0
4,01/01 00:50:00,0.116668,0.0,0.0
...,...,...,...,...
52555,12/31 23:20:00,0.000000,0.0,0.0
52556,12/31 23:30:00,0.000000,0.0,0.0
52557,12/31 23:40:00,0.158442,0.0,0.0
52558,12/31 23:50:00,0.197023,0.0,0.0
